# Imports

In [217]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Read Data

In [218]:
df_train = pd.read_csv('X_train.csv', encoding='cp949')
df_test = pd.read_csv('X_test.csv', encoding='cp949')
y_train = pd.read_csv('y_train.csv').gender
IDtest = df_test.custid.unique()

df_train.head()

,custid,sales_date,sales_time,str_nm,goodcd,brd_nm,corner_nm,pc_nm,part_nm,team_nm,buyer_nm,import_flg,tot_amt,dis_amt,net_amt,inst_mon,inst_fee
0,0,2000-06-25 00:00:00,1212,무역점,2116050008000,에스티로더,수입종합화장품,화장품,명품잡화,잡화가용팀,화장품,1,90000,9000,81000,3,0
1,0,2000-06-25 00:00:00,1242,무역점,4125440008000,시슬리,수입종합화장품,화장품,명품잡화,잡화가용팀,화장품,1,39000,3900,35100,1,0
2,0,2000-08-26 00:00:00,1810,본점,2116052008000,크리니크,수입종합화장품,화장품,잡화파트,잡화가용팀,화장품,1,175000,17500,157500,3,0
3,0,2000-08-26 00:00:00,1830,본점,4106430119900,듀퐁,수입의류,명품토탈,잡화파트,잡화가용팀,수입명품,1,455000,45500,409500,3,0
4,0,2000-09-03 00:00:00,1802,무역점,2139141008000,랑콤,수입종합화장품,화장품,명품잡화,잡화가용팀,화장품,0,100000,10000,90000,3,0


#### 판매데이터 구조
- custid: 고객ID
- sales_date: 판매일자 (문자)
- str_nm: 판매시간 (4자리 숫자)
- goodcd: 상품코드
- brd_nm: 브랜드명
- corner_nm: 코너명
- pc_nm: PC명
- part_nm: 파트명
- team_nm: 판매팀명
- buyer_nm: Buying MD
- import_flg: 수입상품여부
- tot_amt: 판매금액
- dis_amt: 할인금액
- net_amt: 판매금액 - 할인금액
- inst_mon: 할부개월수

<p>*`백화점 영업조직`: 306개 corner < 76개 pc < 29개 part < 3개 team*</p>
<p>*`y_train.csv`에서 target값 0은 남자, 1은 여자를 나타냄*</p>

# Make Features

In [238]:
#총 구매액
train_feature=df_train.groupby('custid')['tot_amt'].agg([('총구매액', 'sum')]).reset_index()
train_feature

# 총 구매건수
f1=df_train.groupby('custid')['tot_amt'].agg([('구매건수', 'size')]).reset_index()
train_feature['구매건수']=f1['구매건수']

# 평균구매가격
f1=df_train.groupby('custid')['tot_amt'].agg([('평균구매가격', 'mean')]).reset_index()
train_feature['평균구매가격']=f1['평균구매가격']

#평균 할부개월수
f1=df_train.groupby('custid')['inst_mon'].agg([('평균할부개월수', 'mean')]).reset_index()
train_feature['평균 할부개월수']=f1['평균할부개월수']

#구매브랜드 종류
f1=df_train.groupby('custid')['brd_nm'].agg([('구매브랜드종류', lambda x: x.nunique())]).reset_index()
train_feature['구매브랜드종류']=f1['구매브랜드종류']

#내점일수
f1=df_train.groupby('custid')['sales_date'].agg([('내점일수','nunique')]).reset_index()
train_feature['내점일수']=f1['내점일수']

#수입상품 구매비율
x = df_train[df_train['import_flg'] == 1].groupby('custid').size() / df_train.groupby('custid').size()
f1 = x.reset_index().rename(columns={0: '수입상품_구매비율'}).fillna(0)
train_feature['수입상품_구매비율']=f1['수입상품_구매비율']

#주말방문 비율
def fw(x):
    k = x.dayofweek
    if k <= 4 :
        return('주중_방문')
    else :
        return('주말_방문')    
    
df = df_train.copy()
df = df.drop_duplicates(['custid','sales_date'])

df['week'] = pd.to_datetime(df.sales_date).apply(fw)
df = pd.pivot_table(df, index='custid', columns='week', values='tot_amt', 
                   aggfunc=np.size, fill_value=0).reset_index()
df['주말방문비율'] = ((df.iloc[:,1] / (df.iloc[:,1]+df.iloc[:,2]))*100).apply(round, args=(1,))
f1 = df.copy().iloc[:,[0,-1]]
train_feature['주말방문비율']=f1['주말방문비율']

#시간대별 구매건수
def f2(x):
    if 9 <= x <= 12 :
        return('아침_구매건수')
    elif 13 <= x <= 17 :
        return('점심_구매건수')
    else :
        return('저녁_구매건수')  # datatime 필드가 시간 형식에 맞지 않은 값을 갖는 경우 저녁시간으로 처리

df_train['timeslot'] = (df_train.sales_time // 100).apply(f2)
f1 = pd.pivot_table(df_train, index='custid', columns='timeslot', values='tot_amt', 
                   aggfunc=np.size, fill_value=0).reset_index()
train_feature=pd.concat([train_feature,f1.iloc[:,[1,2,3]]],axis=1)
train_feature

,custid,총구매액,구매건수,평균구매가격,평균 할부개월수,구매브랜드종류,내점일수,수입상품_구매비율,주말방문비율,아침_구매건수,저녁_구매건수,점심_구매건수
0,0,1742000,11,158363.636364,2.818182,7,7,0.636364,42.9,2,9,0
1,1,2772100,26,106619.230769,2.461538,19,16,0.423077,50.0,7,5,14
2,2,3750850,11,340986.363636,3.454545,7,7,0.090909,28.6,3,4,4
3,3,2300500,30,76683.333333,2.666667,21,13,0.000000,38.5,6,13,11
4,4,1045000,4,261250.000000,4.500000,4,2,0.250000,50.0,0,0,4
5,5,5053759,32,157929.968750,1.875000,21,21,0.187500,23.8,9,12,11
6,6,3785029,31,122097.709677,1.838710,23,11,0.096774,36.4,4,8,19
7,7,1223182,35,34948.057143,1.400000,20,23,0.085714,26.1,0,6,29
8,8,1267500,18,70416.666667,2.111111,13,10,0.111111,70.0,1,3,14
9,9,4956620,59,84010.508475,1.000000,35,34,0.101695,14.7,19,24,16


In [239]:
#방문빈도성
df_train['month']=pd.to_datetime(df_train.sales_date).dt.month
a=df_train.groupby('custid')['month'].agg([('가장 최근 구매 달','max')]).reset_index()
b=df_train.groupby('custid')['month'].agg([('가장 예전 구매 달','min')]).reset_index()
c=df_train.groupby('custid')['month'].agg([('구매를 한 달의 수','nunique')]).reset_index()
a['방문빈도성']=round((a['가장 최근 구매 달']-b['가장 예전 구매 달'])/c['구매를 한 달의 수'],1)
a
train_feature['방문빈도성']=a['방문빈도성']

# 구매상품 다양성
f4=df_train.groupby('custid')['brd_nm'].agg([('구매상품 다양성','nunique')]).reset_index()
f4['구매상품 다양성']=f4['구매상품 다양성']/1191
f4=round(f4,2)
train_feature['구매상품 다양성']=f4['구매상품 다양성']

#최대 할부개월수
a=df_train.groupby('custid')['inst_mon'].agg([('최대할부개월수', 'max')]).reset_index()
a
train_feature['최대할부개월수']=a['최대할부개월수']

# 브랜드 구매수
a=df_train.groupby('custid')['brd_nm'].agg([('구매 브랜드 수','nunique')]).reset_index()
train_feature['구매 브랜드 수']=a['구매 브랜드 수']

#브랜드 편중도 = 해당 고객이 구매한 브랜드 수 / 전체 브랜드 수
train_feature['브랜드 편중도']=a['구매 브랜드 수']/df_train['brd_nm'].nunique()

# 내점당 평균 구매건수 
f2 = df_train.groupby('custid')['sales_date'].agg([('하','value_counts')]).reset_index()
f2= f2.groupby('custid')['하'].agg([('내점당 평균 구매건수','mean')]).reset_index()
f2['내점당 평균 구매건수'] = round(f2['내점당 평균 구매건수'],2)
train_feature['내점당 평균 구매건수']=f2['내점당 평균 구매건수']

# 회원별 구매 집중도

f3 = df_train.groupby('custid')['brd_nm'].agg([('Diver', 'unique')]).reset_index()
f3['다른브랜드']=0

for i in range(0,500):
    f3.iloc[i,2] = f3.iloc[i,1].size

g = df_train.groupby('custid')['brd_nm'].agg( [('브랜드별구매수','value_counts')]).reset_index()
h = g.groupby('custid')['브랜드별구매수'].agg( [('브랜드별최다구매수','max')]).reset_index()
g_h = pd.merge(g,h,on='custid');g_h
g_h.loc[g_h['브랜드별구매수'] == g_h['브랜드별최다구매수'],'최다구매브랜드'] = 'ㅋㅋ' 
g_h = g_h.dropna(how='any') 
g_h = g_h.drop_duplicates(['custid']);g_h

ff = df_train.groupby('custid')['tot_amt'].agg([('총구매횟수','size')]).reset_index()
ff_g_h = pd.merge(ff,g_h,on='custid');ff_g_h 
ff_g_h['구매집중도'] = round((ff_g_h['브랜드별최다구매수']/ff_g_h['총구매횟수'])*100,2)
ff_g_h
ff_g_h = ff_g_h.drop(['브랜드별구매수','총구매횟수','brd_nm','브랜드별최다구매수','최다구매브랜드'],1)
ff_g_h
train_feature['구매집중도']=ff_g_h['구매집중도']

In [240]:
train_feature.shape

(30000, 19)

In [241]:
train_feature.columns

Index(['custid', '총구매액', '구매건수', '평균구매가격', '평균 할부개월수', '구매브랜드종류', '내점일수',
       '수입상품_구매비율', '주말방문비율', '아침_구매건수', '저녁_구매건수', '점심_구매건수', '방문빈도성',
       '구매상품 다양성', '최대할부개월수', '구매 브랜드 수', '브랜드 편중도', '내점당 평균 구매건수', '구매집중도'],
      dtype='object')

# Test Set Make Features

In [225]:
# 총 구매액
test_feature=df_test.groupby('custid')['tot_amt'].agg([('총구매액', 'sum')]).reset_index()
test_feature

# 총 구매건수
a=df_test.groupby('custid')['tot_amt'].agg([('구매건수', 'size')]).reset_index()
test_feature['구매건수']=a['구매건수']

# 평균구매가격
a=df_test.groupby('custid')['tot_amt'].agg([('평균구매가격', 'mean')]).reset_index()
test_feature['평균구매가격']=a['평균구매가격']

#평균 할부개월수
a=df_test.groupby('custid')['inst_mon'].agg([('평균할부개월수', 'mean')]).reset_index()
test_feature['평균할부개월수']=a['평균할부개월수']

#구매브랜드 종류
a=df_test.groupby('custid')['brd_nm'].agg([('구매브랜드종류', lambda x: x.nunique())]).reset_index()
test_feature['구매브랜드종류']=a['구매브랜드종류']

#내점일수
a=df_test.groupby('custid')['sales_date'].agg([('내점일수','nunique')]).reset_index()
test_feature['내점일수']=a['내점일수']

#수입상품 구매비율
x = df_test[df_test['import_flg'] == 1].groupby('custid').size() / df_test.groupby('custid').size()
f = x.reset_index().rename(columns={0: '수입상품_구매비율'}).fillna(0)
f.iloc[:,1] = (f.iloc[:,1]*100).apply(round, args=(1,))
test_feature['수입상품_구매비율']=f['수입상품_구매비율']

#주말방문 비율
def fw(x):
    k = x.dayofweek
    if k <= 4 :
        return('주중_방문')
    else :
        return('주말_방문')    
    
df = df_test.copy()
df = df.drop_duplicates(['custid','sales_date'])

df['week'] = pd.to_datetime(df.sales_date).apply(fw)
df = pd.pivot_table(df, index='custid', columns='week', values='tot_amt', 
                   aggfunc=np.size, fill_value=0).reset_index()
df['주말방문비율'] = ((df.iloc[:,1] / (df.iloc[:,1]+df.iloc[:,2]))*100).apply(round, args=(1,))
f = df.copy().iloc[:,[0,-1]]
test_feature['주말방문비율']=f['주말방문비율']

#시간대별 구매건수
def f2(x):
    if 9 <= x <= 12 :
        return('아침_구매건수')
    elif 13 <= x <= 17 :
        return('점심_구매건수')
    else :
        return('저녁_구매건수')  # datatime 필드가 시간 형식에 맞지 않은 값을 갖는 경우 저녁시간으로 처리

df_test['timeslot'] = (df_test.sales_time // 100).apply(f2)
f = pd.pivot_table(df_test, index='custid', columns='timeslot', values='tot_amt', 
                   aggfunc=np.size, fill_value=0).reset_index()
test_feature=pd.merge(test_feature,f,on='custid')


#방문빈도성
df_test['month']=pd.to_datetime(df_test.sales_date).dt.month
a=df_test.groupby('custid')['month'].agg([('가장 최근 구매 달','max')]).reset_index()
b=df_test.groupby('custid')['month'].agg([('가장 예전 구매 달','min')]).reset_index()
c=df_test.groupby('custid')['month'].agg([('구매를 한 달의 수','nunique')]).reset_index()
a['방문빈도성']=round((a['가장 최근 구매 달']-b['가장 예전 구매 달'])/c['구매를 한 달의 수'],1)
a
test_feature['방문빈도성']=a['방문빈도성']

# 구매상품 다양성
f4=df_test.groupby('custid')['brd_nm'].agg([('구매상품 다양성','nunique')]).reset_index()
f4['구매상품 다양성']=f4['구매상품 다양성']/1191
f4=round(f4,2)
test_feature['구매상품 다양성']=f4['구매상품 다양성']

In [226]:
#최대 할부개월수
a=df_test.groupby('custid')['inst_mon'].agg([('최대할부개월수', 'max')]).reset_index()
a
test_feature['최대할부개월수']=a['최대할부개월수']

# 브랜드 구매수
a=df_test.groupby('custid')['brd_nm'].agg([('구매 브랜드 수','nunique')]).reset_index()
test_feature['구매 브랜드 수']=a['구매 브랜드 수']

#브랜드 편중도 = 해당 고객이 구매한 브랜드 수 / 전체 브랜드 수
test_feature['브랜드 편중도']=test_feature['구매 브랜드 수']/df_test['brd_nm'].nunique()

# 내점당 평균 구매건수 

f = df_test.groupby('custid')['sales_date'].agg([('하','value_counts')]).reset_index()
f= f.groupby('custid')['하'].agg([('내점당 평균 구매건수','mean')]).reset_index()
f['내점당 평균 구매건수'] = round(f['내점당 평균 구매건수'],2)
test_feature['내점당 평균 구매건수']=f['내점당 평균 구매건수']

# 회원별 구매 집중도
f = df_test.groupby('custid')['brd_nm'].agg([('Diver', 'unique')]).reset_index()
f['다른브랜드']=0

for i in range(0,500):
    f.iloc[i,2] = f.iloc[i,1].size

g = df_test.groupby('custid')['brd_nm'].agg( [('브랜드별구매수','value_counts')]).reset_index()
h = g.groupby('custid')['브랜드별구매수'].agg( [('브랜드별최다구매수','max')]).reset_index()
g_h = pd.merge(g,h,on='custid');g_h
g_h.loc[g_h['브랜드별구매수'] == g_h['브랜드별최다구매수'],'최다구매브랜드'] = 'ㅋㅋ' 
g_h = g_h.dropna(how='any') 
g_h = g_h.drop_duplicates(['custid']);g_h

ff = df_test.groupby('custid')['tot_amt'].agg([('총구매횟수','size')]).reset_index()
ff_g_h = pd.merge(ff,g_h,on='custid');ff_g_h 
ff_g_h['구매집중도'] = round((ff_g_h['브랜드별최다구매수']/ff_g_h['총구매횟수'])*100,2)
ff_g_h
ff_g_h = ff_g_h.drop(['브랜드별구매수','총구매횟수','brd_nm','브랜드별최다구매수','최다구매브랜드'],1)
ff_g_h
test_feature['구매집중도']=ff_g_h['구매집중도']

In [229]:
test_feature.shape

(19995, 19)

In [231]:
test_feature.columns

Index(['custid', '총구매액', '구매건수', '평균구매가격', '평균할부개월수', '구매브랜드종류', '내점일수',
       '수입상품_구매비율', '주말방문비율', '아침_구매건수', '저녁_구매건수', '점심_구매건수', '방문빈도성',
       '구매상품 다양성', '최대할부개월수', '구매 브랜드 수', '브랜드 편중도', '내점당 평균 구매건수', '구매집중도'],
      dtype='object')

# Transform Data with One-hot Encoding

In [ ]:
df_train = df_train.drop(['sales_date','str_nm','brd_nm','corner_nm','pc_nm','part_nm','team_nm','buyer_nm','season','timeslot','주구매코너_가정용품','주구매코너_가정용품파트','주구매코너_골프/유니캐쥬얼','주구매코너_공산품','주구매코너_공산품파트','주구매코너_남성의류' , '주구매코너_남성정장스포츠','주구매코너_로얄부띠끄','주구매코너_로얄부틱','주구매코너_명품잡화','주구매코너_생식품','주구매코너_생식품파트','주구매코너_스포츠캐주얼','주구매코너_스포츠캐쥬얼','주구매코너_아동','주구매코너_아동,스포츠' , '주구매코너_아동문화' , '주구매코너_여성의류파트' , '주구매코너_여성정장' , '주구매코너_여성캐주얼' , '주구매코너_여성캐쥬얼' , '주구매코너_영라이브' ,'주구매코너_영어덜트캐쥬얼' , '주구매코너_영캐릭터','주구매코너_영플라자' , '주구매코너_잡화' , '주구매코너_잡화파트' , '주구매코너_케주얼,구두,아동' , '주구매코너_패션잡화' , '고객 등급' , '구매 제품 성향 변수' , '분기별' , '소비추세(구매건수)' ],1)

df_test = df_test.drop(['sales_date','str_nm','brd_nm','corner_nm','pc_nm','part_nm','team_nm','buyer_nm','season','timeslot','주구매코너_가정용품','주구매코너_가정용품파트','주구매코너_골프/유니캐쥬얼','주구매코너_공산품','주구매코너_공산품파트','주구매코너_남성의류' , '주구매코너_남성정장스포츠','주구매코너_로얄부띠끄','주구매코너_로얄부틱','주구매코너_명품잡화','주구매코너_생식품','주구매코너_생식품파트','주구매코너_스포츠캐주얼','주구매코너_스포츠캐쥬얼','주구매코너_아동','주구매코너_아동,스포츠' , '주구매코너_아동문화' , '주구매코너_여성의류파트' , '주구매코너_여성정장' , '주구매코너_여성캐주얼' , '주구매코너_여성캐쥬얼' , '주구매코너_영라이브' ,'주구매코너_영어덜트캐쥬얼' , '주구매코너_영캐릭터','주구매코너_영플라자' , '주구매코너_잡화' , '주구매코너_잡화파트' , '주구매코너_케주얼,구두,아동' , '주구매코너_패션잡화' , '고객 등급' , '구매 제품 성향 변수' , '분기별' , '소비추세(구매건수)' ],1)

df_all = pd.concat([df_train, df_test])
X_train = pd.pivot_table(df_all, index='custid', columns=level, values='tot_amt',
                         aggfunc=lambda x: np.where(len(x) >=1, 1, 0), fill_value=0). \
                         reset_index(). \
                         query('custid not in @IDtest'). \
                         drop(columns=['custid']).values
X_test = pd.pivot_table(df_all, index='custid', columns=level, values='tot_amt',
                         aggfunc=lambda x: np.where(len(x) >=1, 1, 0), fill_value=0). \
                         reset_index(). \
                         query('custid in @IDtest'). \
                         drop(columns=['custid']).values

max_features = X_train.shape[1]

# Build Models

In [ ]:
# Learn XGB
from xgboost import XGBClassifier
import sys, warnings
if not sys.warnoptions: warnings.simplefilter("ignore")

model = XGBClassifier(random_state=0, n_jobs=-1)
model.fit(X_train, y_train)

# Make Submissions

In [ ]:
pred = model.predict_proba(X_test)[:,1]
fname = 'submissions_OHE.csv'
submissions = pd.concat([pd.Series(IDtest, name="custid"), pd.Series(pred, name="gender")] ,axis=1)
submissions.to_csv(fname, index=False)
print("'{}' is ready to submit." .format(fname))

# End